# Atividade 01 - Criando objetos MNE

Dennis Felipe Urtubia e Pedro Perozin

# Carregamento dos dados, labels e descritor

In [15]:
import numpy as np
import json
import mne

data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)

Estruturas => dados (125, 257, 1205) labels (125,)


# Criação do EpochsArray

In [26]:
data = data[:,:256,:]
print(data.shape)

trial_duration = 5
sampling_frequency = data.shape[-1] / trial_duration

ch_names = data.shape[1]
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(ch_names, sampling_frequency, ch_types)

events = np.array([[index, 0, event] for index, event in enumerate(labels)])

# por fim a criação do EpochsArray
epoch_array = mne.EpochsArray(data, info, events)


# TODO setar set_montage
kind = 'GSN-HydroCel-256'
epoch_array.set_montage(kind)

(125, 256, 1205)
125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


ValueError: DigMontage is a only a subset of info. There are 256 channel positions not present in the DigMontage. The required channels are: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '1

In [19]:
epoch_array.events

array([[  0,   0,   4],
       [  1,   0,   2],
       [  2,   0,   3],
       [  3,   0,   5],
       [  4,   0,   1],
       [  5,   0,   2],
       [  6,   0,   5],
       [  7,   0,   4],
       [  8,   0,   2],
       [  9,   0,   3],
       [ 10,   0,   1],
       [ 11,   0,   5],
       [ 12,   0,   4],
       [ 13,   0,   3],
       [ 14,   0,   2],
       [ 15,   0,   4],
       [ 16,   0,   1],
       [ 17,   0,   2],
       [ 18,   0,   5],
       [ 19,   0,   3],
       [ 20,   0,   4],
       [ 21,   0,   1],
       [ 22,   0,   3],
       [ 23,   0,   1],
       [ 24,   0,   3],
       [ 25,   0,   4],
       [ 26,   0,   2],
       [ 27,   0,   3],
       [ 28,   0,   5],
       [ 29,   0,   1],
       [ 30,   0,   2],
       [ 31,   0,   5],
       [ 32,   0,   4],
       [ 33,   0,   2],
       [ 34,   0,   3],
       [ 35,   0,   1],
       [ 36,   0,   5],
       [ 37,   0,   4],
       [ 38,   0,   3],
       [ 39,   0,   2],
       [ 40,   0,   4],
       [ 41,   0